In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
# Needs to be installed in an environment with CUDA available
!pip install inplace-abn
# Restart notebook after installation

Defaulting to user installation because normal site-packages is not writeable


In [39]:
from pathlib import Path
from src.models.tresnet.tresnet import TResNet
import torch


print(f"Num classes: {num_classes}")

Num classes: 9605


In [40]:
checkpoint_path = Path("checkpoints/mtresnet_opim_86.72.pth")

def build_tresnet_m_model(pretrained: bool = True):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if pretrained:
        state = torch.load(checkpoint_path, map_location=device)
        num_classes = state["num_classes"]
    else:
        num_classes = 1
    
    model = TResNet(
        layers=[3, 4, 11, 3],
        num_classes=num_classes,
        in_chans=3,
        do_bottleneck_head=True
    )
    if pretrained:
        model.load_state_dict(state["model"], strict=True)

    return nn.Sequential(
        model.body.SpaceToDepth,
        model.body.conv1,
        model.body.layer1,
        model.body.layer2,
        model.body.layer3,
        model.body.layer4[0]
    )

In [41]:
import torch.nn as nn

model = build_tresnet_m_model(pretrained=True)

In [35]:
from torchinfo import summary

In [42]:
summary(model, input_size=(1, 3, 1024, 1024), depth=5)

Layer (type:depth-idx)                                  Output Shape              Param #
Sequential                                              [1, 2048, 32, 32]         --
├─SpaceToDepthModule: 1-1                               [1, 48, 256, 256]         --
├─Sequential: 1-2                                       [1, 64, 256, 256]         --
│    └─Conv2d: 2-1                                      [1, 64, 256, 256]         27,648
│    └─InPlaceABN: 2-2                                  [1, 64, 256, 256]         128
├─Sequential: 1-3                                       [1, 64, 256, 256]         --
│    └─BasicBlock: 2-3                                  [1, 64, 256, 256]         --
│    │    └─Sequential: 3-1                             [1, 64, 256, 256]         --
│    │    │    └─Conv2d: 4-1                            [1, 64, 256, 256]         36,864
│    │    │    └─InPlaceABN: 4-2                        [1, 64, 256, 256]         128
│    │    └─Sequential: 3-2                       